# Apply Cell Health models to cmQTL morphology profiles

Previously, we trained machine learning models to predict 70 different cell health phenotypes from Cell Painting morphology profiles ([Way et al. 2020](https://doi.org/10.1101/2020.07.08.193938)). Here, we apply three of the models to well-level aggregate profiles to separate data from the cmQTL project.

## Cell Health models

The three models include:

1. ALL - # cells (`cc_all_n_objects`)
2. ALL - Nucleus Area (`cc_all_nucleus_area_mean`)
3. % Dead Only (CASP-; DRAQ7+) (`vb_percent_dead_only`)

## Input

Input to this notebook are normalized morphology profiles (prior to feature selection) for the cmQTL dataset.

## Output

Predictions for the three cell health models per profile.

## Discussion

See https://github.com/broadinstitute/cmQTL/issues/47 for more details.

## WARNING

Currently, no predictions are output - we are working towards resolving the missing feature problem.
There are 506 features measured in the Cell Health project not measured in cmQTL.

In [1]:
import pathlib
import pandas as pd
from joblib import load

from pycytominer.cyto_utils import infer_cp_features

## Load and process data

In [2]:
profile_file = pathlib.Path("data/cell-health/profiles/anycells.well.forCellHealth.tab")
profile_df = pd.read_csv(profile_file, sep="\t")

print(profile_df.shape)
profile_df.head(2)

(2160, 3582)


,Metadata_line_ID,Metadata_Plate,Metadata_Well,Cells_AreaShape_Area,Cells_AreaShape_Center_X,Cells_AreaShape_Center_Y,Cells_AreaShape_Center_Z,Cells_AreaShape_Compactness,Cells_AreaShape_Eccentricity,Cells_AreaShape_EulerNumber,...,Metadata_donor_Source,Metadata_donor_Sex,Metadata_donor_Race,Metadata_donor_Ethnicity,Metadata_donor_Family,Metadata_donor_BSP.Collab.Participant.ID,Metadata_donor_BSP.Collab.Sample.ID,Metadata_Doubling_Hour,Metadata_donor_Age,Metadata_donor_disease.name
0,141,BR00106708,B01,-0.045850,-0.935342,-0.057469,0,-1.216746,-0.973771,0.968194,...,PBMC,Female,Other,Not Hispanic or Latino,CIRM00074,CW60132,CW60132_P13_MT_1-18-18,40.95,32.0,CONTROL
1,141,BR00106708,E10,-0.562669,0.566752,0.252718,0,-1.550861,-1.317730,1.151263,...,PBMC,Female,Other,Not Hispanic or Latino,CIRM00074,CW60132,CW60132_P13_MT_1-18-18,40.95,32.0,CONTROL


In [3]:
! md5 $profile_file

MD5 (data/cell-health/profiles/anycells.well.forCellHealth.tab) = 582c803407d129bff383c414ad32127e


In [4]:
# Load dataset used for training to get order of coefficients
commit = "07e4b40c39dd27084be36fbef4d64c5654b2960f"
data_url = f"https://github.com/broadinstitute/cell-health/raw/{commit}/3.train/data/x_train_modz.tsv.gz"
cell_health_df = pd.read_csv(data_url, sep="\t")

cell_health_feature_order = infer_cp_features(cell_health_df)

print(cell_health_df.shape)
cell_health_df.head(2)

(303, 952)


,Metadata_profile_id,Metadata_cell_line,Metadata_pert_name,Cells_AreaShape_Center_Y,Cells_AreaShape_Compactness,Cells_AreaShape_Eccentricity,Cells_AreaShape_Extent,Cells_AreaShape_Orientation,Cells_AreaShape_Zernike_0_0,Cells_AreaShape_Zernike_1_1,...,Nuclei_Texture_SumEntropy_RNA_5_0,Nuclei_Texture_SumVariance_AGP_20_0,Nuclei_Texture_SumVariance_AGP_5_0,Nuclei_Texture_SumVariance_DNA_10_0,Nuclei_Texture_SumVariance_DNA_20_0,Nuclei_Texture_SumVariance_DNA_5_0,Nuclei_Texture_Variance_AGP_5_0,Nuclei_Texture_Variance_DNA_10_0,Nuclei_Texture_Variance_DNA_20_0,Nuclei_Texture_Variance_DNA_5_0
0,profile_340,HCC44,RHOA-2,0.029292,0.607142,-0.576461,-2.459725,0.587515,-1.183370,0.257162,...,-2.964612,-2.131689,-2.073252,-2.623642,-2.597662,-2.869573,-2.367492,-3.110212,-3.150354,-2.901882
1,profile_6,A549,ATP50-1,-0.087900,0.186519,0.562200,-0.060381,-0.206748,-0.318174,0.682207,...,-0.030618,-0.285945,-0.297946,-0.254188,-0.020307,-0.190692,-0.358839,-0.388787,-0.318564,-0.311846


In [5]:
cp_cols = infer_cp_features(profile_df)
metadata_cols = infer_cp_features(profile_df, metadata=True)

# Reindex the cmQTL profiles
profile_reindexed_df = profile_df.reindex(columns=metadata_cols + cell_health_feature_order)

In [6]:
features_missing = set(cell_health_feature_order).difference(set(cp_cols))

print(f"There are a total of {len(cell_health_feature_order)} features in the Cell Health project.")
print(f"There are {len(features_missing)} features in the Cell Health project missing from the cmQTL project.")

There are a total of 949 features in the Cell Health project.
There are 506 features in the Cell Health project missing from the cmQTL project.


## Load models

In [7]:
model_dir = pathlib.Path("data/cell-health/models/")
model_dict = {
    "cc_all_n_objects": {"name": "ALL - # cells"},
    "cc_all_nucleus_area_mean": {"name": "ALL - Nucleus Area"},
    "vb_percent_dead_only": {"name": "% Dead Only (CASP-; DRAQ7+)"}
}

In [8]:
for model in model_dict.keys():
    model_file = model_dir / pathlib.Path(f"cell_health_modz_target_{model}_shuffle_False_transform_raw.joblib")
    model_job = load(model_file)
    
    model_coef_df = (
        pd.DataFrame(
            model_job.coef_, columns=[model], index=cell_health_feature_order
        )
        .reset_index()
        .rename({"index": "cp_features"}, axis="columns")
        .assign(missing=False)
    )
    model_coef_df.loc[model_coef_df.cp_features.isin(features_missing), "missing"] = True
    model_coef_df = model_coef_df.assign(abs_coef=model_coef_df.loc[:, model].abs())
    
    model_dict[model]["model"] = model_job
    model_dict[model]["coef"] = model_coef_df
    
    # Check the total contribution lost by missing features
    print(model_coef_df.groupby(["missing"])["abs_coef"].sum())

missing
False    0.987737
True     0.523663
Name: abs_coef, dtype: float64
missing
False    2.821468
True     2.532892
Name: abs_coef, dtype: float64
missing
False    1.429959
True     1.065650
Name: abs_coef, dtype: float64


In [9]:
features_missing

{'Cells_Correlation_Correlation_DNA_AGP',
 'Cells_Correlation_Correlation_DNA_ER',
 'Cells_Correlation_Correlation_DNA_Mito',
 'Cells_Correlation_Correlation_DNA_RNA',
 'Cells_Correlation_Correlation_ER_AGP',
 'Cells_Correlation_Correlation_ER_RNA',
 'Cells_Correlation_Correlation_Mito_AGP',
 'Cells_Correlation_Correlation_Mito_ER',
 'Cells_Correlation_Correlation_Mito_RNA',
 'Cells_Correlation_Correlation_RNA_AGP',
 'Cells_Correlation_K_AGP_DNA',
 'Cells_Correlation_K_AGP_ER',
 'Cells_Correlation_K_AGP_Mito',
 'Cells_Correlation_K_AGP_RNA',
 'Cells_Correlation_K_ER_AGP',
 'Cells_Correlation_K_ER_DNA',
 'Cells_Correlation_K_Mito_AGP',
 'Cells_Correlation_K_Mito_DNA',
 'Cells_Correlation_K_Mito_ER',
 'Cells_Correlation_K_Mito_RNA',
 'Cells_Correlation_K_RNA_AGP',
 'Cells_Correlation_K_RNA_DNA',
 'Cells_Correlation_Overlap_DNA_AGP',
 'Cells_Correlation_Overlap_DNA_RNA',
 'Cells_Correlation_Overlap_ER_AGP',
 'Cells_Correlation_Overlap_ER_RNA',
 'Cells_Correlation_Overlap_Mito_AGP',
 'Cell